<h1 align="center"><font color='Red'>Assignment for Data QA & QC Internship @ Datahut</font></h1>

#### Name: Rishana
#### Organization: Datahut
#### Date: 21.06.2024

#### <b><font color='brown'>IMPORTING MODULES:</font></b>

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
import math
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#### <b><font color='brown'>LOADING DATASET:</font></b>

In [ ]:
#loading data
df=pd.read_csv('messy_data.csv')
df

#### <b><font color='brown'>DATA EXPLORATION</font></b>

In [ ]:
# To get info
df.info()

In [ ]:
df.shape

In [ ]:
#To display the initial rows
df.head()

In [ ]:
#To display the final rows of the DataFrame
df.tail()

In [ ]:
#To display column labels of DataFrame
df.columns

In [ ]:
# Display summary statistics to identify outliers and inconsistencies
print("\nStatistical analysis")
df.describe(include='all')

In [ ]:
#unique value counts
for column in df.columns:
    unique_values_count = df[column].nunique()
    print(f"Unique values count for {column}: {unique_values_count}")

In [ ]:
# Display the unique values in each column to identify inconsistencies
unique_values = {col: df[col].unique() for col in df.columns}
print(unique_values)

In [ ]:
df['Name'].unique()

#### <b><font color='brown'>DATA PREPROCESSING</font></b>

In [ ]:
# Dropping the unnecessary 'Unnamed: 0' and 'ID' column
# Reason: These column not needed for analysis.
df.drop(columns=['Unnamed: 0', 'ID'], inplace=True)

In [ ]:
#Checking is there any null value
df.isnull().sum()

Most column contains null values

In [ ]:
# Fill missing numerical values with median
df['Age'].fillna(df['Age'].median(), inplace=True)

df['Salary'].fillna(df['Salary'].median(), inplace=True)

# For 'Department', fill missing values with 'Unknown'
df['Department'] = df['Department'].fillna('Unknown')

# For 'Join Date', fill missing values with '1970-01-01'
df['Join Date'] = df['Join Date'].fillna('1970-01-01')

In [ ]:
#ensuring all null values are replaced
df.isnull().sum()

In [ ]:
# Dropping Duplicates
df=df.drop_duplicates()
df=df.reset_index(drop=True)
df

In [ ]:
# Assumption: Valid emails contain "@" and a domain. Invalid emails will be corrected if possible.
def correct_email(email):
    if pd.isna(email):
        return email
    if not re.match(r'.+@.+\..+', email):
        parts = email.split('@')
        if len(parts) == 1:
            return parts[0] + "@example.com"
        elif len(parts) == 2:
            if '.' not in parts[1]:
                return parts[0] + "@" + parts[1] + ".com"
    return email

df['Email'] = df['Email'].apply(correct_email)

In [ ]:
df['Email'].unique()

In [ ]:
#filtering professional emails

#function to check for professional email formats
def is_professional_email(email):
    if pd.isna(email):
        return False
    # Match email against the professional domains
    return bool(re.match(r'.+@.+\.(com|org|net|biz|info)$', email))

# Filter the dataframe to retain only rows with professional emails
df = df[df['Email'].apply(is_professional_email)]

In [ ]:
# Define a function to clean the 'Name' column

#Clean the Name column

# Assumption: Names should only contain alphabetic characters and spaces.Remove non-alphabetic characters at the end of the name and ensure proper capitalization.

def clean_name(name):
    if pd.isna(name):
        return name
    # Remove non-alphabetic characters from the end of the name
    name = re.sub(r'[^a-zA-Z\s]+$', '', name)
    # Ensure proper capitalization
    name = name.title()
    return name

# Apply the clean_name function to the 'Name' column
df['Name'] = df['Name'].astype(str).apply(clean_name)

In [ ]:
df['Name'].unique()

In [ ]:
#Standardize the Join Date column
# Assumption: Dates should follow the format YYYY-MM-DD.
df['Join Date'] = pd.to_datetime(df['Join Date'], errors='coerce')
df['Join Date'] = df['Join Date'].dt.strftime('%Y-%m-%d')

In [ ]:
df['Join Date'].unique()

In [ ]:
#  Standardize Department Names
# Assumption: Known department name variations and typos are mapped to standard names.
department_mapping = {
    'Hr': 'HR',
    'Human Resources': 'HR',
    'Eng': 'Engineering',
    'Engg': 'Engineering',
    'Engine': 'Engineering',
    'Mktg': 'Marketing',
    'Mkt': 'Marketing',
    'Mrkt': 'Marketing',
    'Sales': 'Sales',
    'Support': 'Support',
    'Suppt': 'Support',
    'Cust Support': 'Support',
    'Customer Support': 'Support',
    # Add any other known typos or variations
}

def standardize_department(department):
    return department_mapping.get(department, department)


df['Department'] = df['Department'].apply(standardize_department)

In [ ]:
#outlier detection
sns.distplot(df['Salary'])
plt.show()

In [ ]:
# Identify outliers in 'Salary' using IQR method
Q1 = df['Salary'].quantile(0.25)
Q3 = df['Salary'].quantile(0.75)
IQR = Q3 - Q1

lw = Q1 - 1.5 * IQR
uw = Q3 + 1.5 * IQR

#capping 'salary' column
df['Salary'] = np.where(df['Salary'] < lw, lw, df['Salary'])
df['Salary'] = np.where(df['Salary'] > uw, uw, df['Salary'])

In [ ]:
#save the cleaned dataset as `cleaned_dataset.csv`.
cleaned_file_path = 'cleaned_dataset.csv'
df.to_csv(cleaned_file_path, index=False)

In [ ]:
# Identify outliers in 'Salary' using IQR method
Q1 = df['Salary'].quantile(0.25)
Q3 = df['Salary'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
df = df[(df['Salary'] >= lower_bound) & (df['Salary'] <= upper_bound)]

In [ ]:
sns.distplot(df['Salary'])
plt.show()

In [ ]:
df